In [1]:
import math
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from mlxtend.regressor import StackingRegressor
from xgboost import XGBRegressor, XGBRFRegressor

In [2]:
def read_data(filename):

    data=pd.read_csv('withindata/'+filename)
    data=data.drop(['name', 'version', 'name.1'], axis='columns')

    features=data.drop(['bug'], axis='columns')
    label=data['bug']

    test_features=features.values.reshape(-1,20)
    test_label=label.values.reshape(-1,1)

    return test_features, test_label

In [3]:
def mean_relative_error(actual, predicted):
    rel_err=[]
    actual,predicted=np.array(actual), np.array(predicted)
    for i in range(0,len(actual)):
        err=abs(actual[i]-predicted[i])/(actual[i]+1)
        rel_err.append(err)
    return np.mean(rel_err)

In [32]:
def run_techniques(filename): 
    
    features, label = read_data(filename)
    train_features, test_features, train_label, test_label = train_test_split(features, label, test_size=0.2)

    stravgmre,stravgmretest=[],[]
    stravgmae,stravgmaetest=[],[]
    stravgrmse,stravgrmsetest=[],[]
    
    stravgmre2,stravgmretest2=[],[]
    stravgmae2,stravgmaetest2=[],[]
    stravgrmse2,stravgrmsetest2=[],[]

    dtravgmre,dtravgmretest=[],[]
    dtravgmae,dtravgmaetest=[],[]
    dtravgrmse,dtravgrmsetest=[],[]

    rfravgmre,rfravgmretest=[],[]
    rfravgmae,rfravgmaetest=[],[]
    rfravgrmse,rfravgrmsetest=[],[]

    etravgmre,etravgmretest=[],[]
    etravgmae,etravgmaetest=[],[]
    etravgrmse,etravgrmsetest=[],[]

    knnavgmre,knnavgmretest=[],[]
    knnavgmae,knnavgmaetest=[],[]
    knnavgrmse,knnavgrmsetest=[],[]

    svravgmre,svravgmretest=[],[]
    svravgmae,svravgmaetest=[],[]
    svravgrmse,svravgrmsetest=[],[]

    bavgmre,bavgmretest=[],[]
    bavgmae,bavgmaetest=[],[]
    bavgrmse,bavgrmsetest=[],[]

    import warnings
    warnings.filterwarnings("ignore")

    for run in range(0,30):
        print('run: ', run+1)

        kf=KFold(n_splits=10,shuffle=True)
        i=1

        strmre,strmretest=[],[]
        strmae,strmaetest=[],[]
        strrmse,strrmsetest=[],[]
        
        strmre2,strmretest2=[],[]
        strmae2,strmaetest2=[],[]
        strrmse2,strrmsetest2=[],[]

        dtrmre,dtrmretest=[],[]
        dtrmae,dtrmaetest=[],[]
        dtrrmse,dtrrmsetest=[],[]

        rfrmre,rfrmretest=[],[]
        rfrmae,rfrmaetest=[],[]
        rfrrmse,rfrrmsetest=[],[]

        etrmre,etrmretest=[],[]
        etrmae,etrmaetest=[],[]
        etrrmse,etrrmsetest=[],[]

        knnmre,knnmretest=[],[]
        knnmae,knnmaetest=[],[]
        knnrmse,knnrmsetest=[],[]

        svrmre,svrmretest=[],[]
        svrmae,svrmaetest=[],[]
        svrrmse,svrrmsetest=[],[]

        bmre,bmretest=[],[]
        bmae,bmaetest=[],[]
        brmse,brmsetest=[],[]

        for train_index,test_index in kf.split(train_features):
            print('fold number: ', i)

            x_train,x_test=train_features[train_index],train_features[test_index]
            y_train,y_test=train_label[train_index],train_label[test_index]
            
            params_tree = {
                'n_estimators':[50,126],
                'min_samples_leaf':[30,50],
                'max_depth':[20,50]
                }
            
            params_d_tree = {
                'max_depth':[20,50]
                }
            
            params_knn = {
                'n_neighbors':[5,10]
                }
            
            params_svr ={
                'C':[5,10],
                'gamma':[0.001,0.1,1]
                }
            
            params_xgb = {
                'max_depth':[10,50],
                'min_child_weight':[1,6]
                }
            
            params_1 = {
                'svr__C':[0.001,0.1,1,5,10],
                'svr__gamma':[0.001, 0.1, 0.5,1],
                'extratreesregressor__n_estimators':[50,100,126,200],
                'extratreesregressor__min_samples_leaf':[20,30,50,10],
                'kneighborsregressor__n_neighbors':[5,7,10,15],
                'randomforestregressor__max_depth':[10,20,30,50],
                'randomforestregressor__n_estimators':[50,100,126,300],
                'meta_regressor__C':[0.001,0.1,1,5,10],
                'meta_regressor__gamma':[0.001,0.1,0.5,1],
                'meta_regressor__kernel':['rbf','linear']
                }
            
            params_2 = {
                'svr__C':[0.001,0.1,1,5,10],
                'svr__gamma':[0.001, 0.1, 0.5,1],
                'extratreesregressor__n_estimators':[50,100,126,200],
                'extratreesregressor__min_samples_leaf':[20,30,50,10],
                'kneighborsregressor__n_neighbors':[5,7,10,15],
                'randomforestregressor__max_depth':[10,20,30,50],
                'randomforestregressor__n_estimators':[50,100,126,300],
                'meta_regressor__max_depth':[10,20,30,50],
                'meta_regressor__n_estimators':[20,50,100,150,500]
                }
            
            print('DTR')
            rand_search_dtr = RandomizedSearchCV(DecisionTreeRegressor(), params_d_tree, cv=5)
            rand_search_dtr.fit(x_train, y_train)
            best_tuned_dtr = rand_search_dtr.best_estimator_
            y_pred = best_tuned_dtr.predict(test_features)
            test_pred = best_tuned_dtr.predict(x_test)
            #testing errors
            dtrmretest.append(mean_relative_error(y_test,test_pred))
            dtrmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
            dtrrmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
            #validation errors
            dtrmre.append(mean_relative_error(test_label,y_pred))
            dtrmae.append(metrics.mean_absolute_error(test_label,y_pred))
            dtrrmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))
            
            print('RFR')
            rand_search_rfr = RandomizedSearchCV(RandomForestRegressor(), params_tree, cv=5)
            rand_search_rfr.fit(x_train, y_train)
            best_tuned_rfr = rand_search_rfr.best_estimator_
            y_pred = best_tuned_rfr.predict(test_features)
            test_pred = best_tuned_rfr.predict(x_test)
            #testing errors
            rfrmretest.append(mean_relative_error(y_test,test_pred))
            rfrmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
            rfrrmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
            #validation errors
            rfrmre.append(mean_relative_error(test_label,y_pred))
            rfrmae.append(metrics.mean_absolute_error(test_label,y_pred))
            rfrrmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))
            
            print('ETR')
            rand_search_etr = RandomizedSearchCV(ExtraTreesRegressor(), params_tree, cv=5)
            rand_search_etr.fit(x_train, y_train)
            best_tuned_etr = rand_search_etr.best_estimator_
            y_pred = best_tuned_etr.predict(test_features)
            test_pred = best_tuned_etr.predict(x_test)
            #testing errors
            etrmretest.append(mean_relative_error(y_test,test_pred))
            etrmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
            etrrmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
            #validation errors
            etrmre.append(mean_relative_error(test_label,y_pred))
            etrmae.append(metrics.mean_absolute_error(test_label,y_pred))
            etrrmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))
            
            print('KNN')
            rand_search_knn = RandomizedSearchCV(KNeighborsRegressor(), params_knn, cv=5)
            rand_search_knn.fit(x_train, y_train)
            best_tuned_knn = rand_search_knn.best_estimator_
            y_pred = best_tuned_knn.predict(test_features)
            test_pred = best_tuned_knn.predict(x_test)
            #testing errors
            knnmretest.append(mean_relative_error(y_test,test_pred))
            knnmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
            knnrmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
            #validation errors
            knnmre.append(mean_relative_error(test_label,y_pred))
            knnmae.append(metrics.mean_absolute_error(test_label,y_pred))
            knnrmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))
            
            print('SVR')
            rand_search_svr = RandomizedSearchCV(SVR(), params_svr, cv=5)
            rand_search_svr.fit(x_train, y_train)
            best_tuned_svr = rand_search_svr.best_estimator_
            y_pred = best_tuned_svr.predict(test_features)
            test_pred = best_tuned_svr.predict(x_test)
            #testing errors
            svrmretest.append(mean_relative_error(y_test,test_pred))
            svrmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
            svrrmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
            #validation errors
            svrmre.append(mean_relative_error(test_label,y_pred))
            svrmae.append(metrics.mean_absolute_error(test_label,y_pred))
            svrrmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))
            
#             print('XGB')
#             rand_search_xgb = RandomizedSearchCV(XGBRegressor(silent=True), params_xgb, cv=5, n_jobs = 4)
#             print('fittinh')
#             rand_search_xgb.fit(x_train, y_train)
#             print('predicting')
#             best_tuned_xgb = rand_search_xgb.best_estimator_
#             y_pred = best_tuned_xgb.predict(test_features)
#             test_pred = best_tuned_xgb.predict(x_test)
#             #testing errors
#             bmretest.append(mean_relative_error(y_test,test_pred))
#             bmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
#             brmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
#             #validation errors
#             bmre.append(mean_relative_error(test_label,y_pred))
#             bmae.append(metrics.mean_absolute_error(test_label,y_pred))
#             brmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))
            
            print('SR')
            reg1=SVR()
            reg2=ExtraTreesRegressor()
            reg3=KNeighborsRegressor()
            reg4=RandomForestRegressor()
            rand_search_sr = RandomizedSearchCV(StackingRegressor(store_train_meta_features=True,regressors=[reg1,reg2,reg3,reg4], meta_regressor=SVR()), params, cv =5)
            print('fitting')
            rand_search_sr.fit(x_train, y_train)
            print('predicting')
            best_tuned_sr = rand_search_sr.best_estimator_
            print(best_tuned_sr.train_meta_features_[0])
            y_pred = best_tuned_sr.predict(test_features)
            test_pred = best_tuned_sr.predict(x_test)
            #testing errors
            strmretest.append(mean_relative_error(y_test,test_pred))
            strmaetest.append(metrics.mean_absolute_error(y_test,test_pred))
            strrmsetest.append(math.sqrt(metrics.mean_squared_error(y_test,test_pred)))
            #validation errors
            strmre.append(mean_relative_error(test_label,y_pred))
            strmae.append(metrics.mean_absolute_error(test_label,y_pred))
            strrmse.append(math.sqrt(metrics.mean_squared_error(test_label,y_pred)))

            i=i+1

        dtravgmre.append(np.mean(dtrmre))
        dtravgmae.append(np.mean(dtrmae))
        dtravgrmse.append(np.mean(dtrrmse))
        dtravgmretest.append(np.mean(dtrmretest))
        dtravgmaetest.append(np.mean(dtrmaetest))
        dtravgrmsetest.append(np.mean(dtrrmsetest))

        rfravgmre.append(np.mean(rfrmre))
        rfravgmae.append(np.mean(rfrmae))
        rfravgrmse.append(np.mean(rfrrmse))
        rfravgmretest.append(np.mean(rfrmretest))
        rfravgmaetest.append(np.mean(rfrmaetest))
        rfravgrmsetest.append(np.mean(rfrrmsetest))

        etravgmre.append(np.mean(etrmre))
        etravgmae.append(np.mean(etrmae))
        etravgrmse.append(np.mean(etrrmse))
        etravgmretest.append(np.mean(etrmretest))
        etravgmaetest.append(np.mean(etrmaetest))
        etravgrmsetest.append(np.mean(etrrmsetest))

        knnavgmre.append(np.mean(knnmre))
        knnavgmae.append(np.mean(knnmae))
        knnavgrmse.append(np.mean(knnrmse))
        knnavgmretest.append(np.mean(knnmretest))
        knnavgmaetest.append(np.mean(knnmaetest))
        knnavgrmsetest.append(np.mean(knnrmsetest))

        svravgmre.append(np.mean(svrmre))
        svravgmae.append(np.mean(svrmae))
        svravgrmse.append(np.mean(svrrmse))
        svravgmretest.append(np.mean(svrmretest))
        svravgmaetest.append(np.mean(svrmaetest))
        svravgrmsetest.append(np.mean(svrrmsetest))

#         bavgmre.append(np.mean(bmre))
#         bavgmae.append(np.mean(bmae))
#         bavgrmse.append(np.mean(brmse))
#         bavgmretest.append(np.mean(bmretest))
#         bavgmaetest.append(np.mean(bmaetest))
#         bavgrmsetest.append(np.mean(brmsetest))

        stravgmre.append(np.mean(strmre))
        stravgmae.append(np.mean(strmae))
        stravgrmse.append(np.mean(strrmse))
        stravgmretest.append(np.mean(strmretest))
        stravgmaetest.append(np.mean(strmaetest))
        stravgrmsetest.append(np.mean(strrmsetest))

        print('dtr: mre: ', np.mean(dtrmre), ' mae: ', np.mean(dtrmae), ' rmse: ', np.mean(dtrrmse))
        print('test dtr: mre: ', np.mean(dtrmretest), ' mae: ', np.mean(dtrmaetest), ' rmse: ', np.mean(dtrrmsetest))        
        print('rfr: mre: ', np.mean(rfrmre), ' mae: ', np.mean(rfrmae), ' rmse: ', np.mean(rfrrmse))
        print('test rfr: mre: ', np.mean(rfrmretest), ' mae: ', np.mean(rfrmaetest), ' rmse: ', np.mean(rfrrmsetest))
        print('etr: mre: ', np.mean(etrmre), ' mae: ', np.mean(etrmae), ' rmse: ', np.mean(etrrmse))
        print('test etr: mre: ', np.mean(etrmretest), ' mae: ', np.mean(etrmaetest), ' rmse: ', np.mean(etrrmsetest))
        print('knn: mre: ', np.mean(knnmre), ' mae: ', np.mean(knnmae), ' rmse: ', np.mean(knnrmse))
        print('test knn: mre: ', np.mean(knnmretest), ' mae: ', np.mean(knnmaetest), ' rmse: ', np.mean(knnrmsetest))
        print('svr: mre: ', np.mean(svrmre), ' mae: ', np.mean(svrmae), ' rmse: ', np.mean(svrrmse))
        print('test svr: mre: ', np.mean(svrmretest), ' mae: ', np.mean(svrmaetest), ' rmse: ', np.mean(svrrmsetest))
#         print('xgb: mre: ', np.mean(bmre), ' mae: ', np.mean(bmae), ' rmse: ', np.mean(brmse))
#         print('test xgb: mre: ', np.mean(bmretest), ' mae: ', np.mean(bmaetest), ' rmse: ', np.mean(brmsetest))
        print('str: mre: ', np.mean(strmre), ' mae: ', np.mean(strmae), ' rmse: ', np.mean(strrmse))
        print('test str: mre: ', np.mean(strmretest), ' mae: ', np.mean(strmaetest), ' rmse: ', np.mean(strrmsetest))

    print('-------------------------------------------------------------------------------------------')
    print('avg dtr: mre: ', np.mean(dtravgmre), ' mae: ', np.mean(dtravgmae), ' rmse: ', np.mean(dtravgrmse))
    print('test avg dtr: mre: ', np.mean(dtravgmretest), ' mae: ', np.mean(dtravgmaetest), ' rmse: ', np.mean(dtravgrmsetest))
    print('avg rfr: mre: ', np.mean(rfravgmre), ' mae: ', np.mean(rfravgmae), ' rmse: ', np.mean(rfravgrmse))
    print('test avg rfr: mre: ', np.mean(rfravgmretest), ' mae: ', np.mean(rfravgmaetest), ' rmse: ', np.mean(rfravgrmsetest))
    print('avg etr: mre: ', np.mean(etravgmre), ' mae: ', np.mean(etravgmae), ' rmse: ', np.mean(etravgrmse))
    print('test avg etr: mre: ', np.mean(etravgmretest), ' mae: ', np.mean(etravgmaetest), ' rmse: ', np.mean(etravgrmsetest))
    print('avg knn: mre: ', np.mean(knnavgmre), ' mae: ', np.mean(knnavgmae), ' rmse: ', np.mean(knnavgrmse))
    print('test avg knn: mre: ', np.mean(knnavgmretest), ' mae: ', np.mean(knnavgmaetest), ' rmse: ', np.mean(knnavgrmsetest))
    print('avg svr: mre: ', np.mean(svravgmre), ' mae: ', np.mean(svravgmae), ' rmse: ', np.mean(svravgrmse))
    print('test avg svr: mre: ', np.mean(svravgmretest), ' mae: ', np.mean(svravgmaetest), ' rmse: ', np.mean(svravgrmsetest))
#     print('avg xgb: mre: ', np.mean(bavgmre), ' mae: ', np.mean(bavgmae), ' rmse: ', np.mean(bavgrmse))
#     print('test avg xgb: mre: ', np.mean(bavgmretest), ' mae: ', np.mean(bavgmaetest), ' rmse: ', np.mean(bavgrmsetest))
    print('avg str: mre: ', np.mean(stravgmre), ' mae: ', np.mean(stravgmae), ' rmse: ', np.mean(stravgrmse))
    print('test avg str: mre: ', np.mean(stravgmretest), ' mae: ', np.mean(stravgmaetest), ' rmse: ', np.mean(stravgrmsetest))

    with open('results-'+ os.path.splitext(filename)[0] +'.txt', 'w') as f:
                        f.write("validation results:\n")
                        f.write("mre, mae, rmse\n")
                        f.write("dtr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(dtravgmre),np.std(dtravgmre),
                                                                            np.mean(dtravgmae), np.std(dtravgmae),
                                                                            np.mean(dtravgrmse), np.std(dtravgrmse)))
                        f.write("rfr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(rfravgmre),np.std(rfravgmre),
                                                                            np.mean(rfravgmae), np.std(rfravgmae),
                                                                            np.mean(rfravgrmse), np.std(rfravgrmse)))
                        f.write("etr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(etravgmre),np.std(etravgmre),
                                                                            np.mean(etravgmae), np.std(etravgmae),
                                                                            np.mean(etravgrmse), np.std(etravgrmse)))
                        f.write("knn: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(knnavgmre),np.std(knnavgmre),
                                                                            np.mean(knnavgmae), np.std(knnavgmae),
                                                                            np.mean(knnavgrmse), np.std(knnavgrmse)))
                        f.write("svr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(svravgmre),np.std(svravgmre),
                                                                            np.mean(svravgmae), np.std(svravgmae),
                                                                            np.mean(svravgrmse), np.std(svravgrmse)))
#                         f.write("xgb: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(bavgmre),np.std(bavgmre),
#                                                                             np.mean(bavgmae), np.std(bavgmae),
#                                                                             np.mean(bavgrmse), np.std(bavgrmse)))
                        f.write("str: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(stravgmre),np.std(stravgmre),
                                                                            np.mean(stravgmae), np.std(stravgmae),
                                                                            np.mean(stravgrmse), np.std(stravgrmse)))
                        f.write("testing results:\n")
                        f.write("mre, mae, rmse\n")
                        f.write("dtr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(dtravgmretest),np.std(dtravgmretest),
                                                                            np.mean(dtravgmaetest), np.std(dtravgmaetest),
                                                                            np.mean(dtravgrmsetest), np.std(dtravgrmsetest)))
                        f.write("rfr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(rfravgmretest),np.std(rfravgmretest),
                                                                            np.mean(rfravgmaetest), np.std(rfravgmaetest),
                                                                            np.mean(rfravgrmsetest), np.std(rfravgrmsetest)))
                        f.write("etr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(etravgmretest),np.std(etravgmretest),
                                                                            np.mean(etravgmaetest), np.std(etravgmaetest),
                                                                            np.mean(etravgrmsetest), np.std(etravgrmsetest)))
                        f.write("knn: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(knnavgmretest),np.std(knnavgmretest),
                                                                            np.mean(knnavgmaetest), np.std(knnavgmaetest),
                                                                            np.mean(knnavgrmsetest), np.std(knnavgrmsetest)))
                        f.write("svr: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(svravgmretest),np.std(svravgmretest),
                                                                            np.mean(svravgmaetest), np.std(svravgmaetest),
                                                                            np.mean(svravgrmsetest), np.std(svravgrmsetest)))
#                         f.write("xgb: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(bavgmretest),np.std(bavgmretest),
#                                                                             np.mean(bavgmaetest), np.std(bavgmaetest),
#                                                                             np.mean(bavgrmsetest), np.std(bavgrmsetest)))
                        f.write("str: %s(%s) & %s(%s) & %s(%s)\n"% (np.mean(stravgmretest),np.std(stravgmretest),
                                                                            np.mean(stravgmaetest), np.std(stravgmaetest),
                                                                            np.mean(stravgrmsetest), np.std(stravgrmsetest)))

In [33]:
for filename in os.listdir('withindata'):
    if '.csv' in filename:
        print(filename)
        run_techniques(filename)

ant-1.7.csv
run:  1
fold number:  1
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09989081 0.80649835 0.4        0.22333333]
fold number:  2
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.1005     0.8736967  0.57142857 0.14083433]
fold number:  3
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10000082 0.5693508  2.26666667 0.48666667]
fold number:  4
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10025425 0.65245316 0.46666667 0.26      ]
fold number:  5
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09973785 0.67305466 0.46666667 0.3015873 ]
fold number:  6
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09998899 0.86122718 0.4        0.17460317]
fold number:  7
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.0998531  0.86214672 0.5        0.31340766]
fold number:  8
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09984443 0.71709082 0.28571429 0.28      ]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10050033 0.81612524 0.46666667 0.1984127 ]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitt

ETR
KNN
SVR
SR
fitting
predicting
[0.09979884 0.64629824 0.2        0.28571429]
fold number:  2
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09988183 0.56411587 0.6        0.26220037]
fold number:  3
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10043849 0.7702916  1.57142857 0.66      ]
fold number:  4
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09992676 0.58580693 0.4        0.23809524]
fold number:  5
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10002246 0.7647071  0.2        0.21      ]
fold number:  6
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10033217 0.83915374 0.46666667 0.29566088]
fold number:  7
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09997674 0.85965692 0.26666667 0.38      ]
fold number:  8
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.1000906  0.56722286 0.53333333 0.23015873]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10010574 0.83408216 0.46666667 0.30666667]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10000067 0.71012196 0.5   

ETR
KNN
SVR
SR
fitting
predicting
[0.1005     0.49675508 1.8        0.5679387 ]
fold number:  2
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.1005     0.83393375 0.71428571 0.32      ]
fold number:  3
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.0999225  0.8906476  0.71428571 0.32333333]
fold number:  4
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10014373 0.87859219 0.5        0.19      ]
fold number:  5
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10029645 0.68846093 0.4        0.25873397]
fold number:  6
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09956232 0.65910392 0.4        0.36370648]
fold number:  7
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10001744 0.83735719 0.71428571 0.38      ]
fold number:  8
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10050036 0.60649097 0.28571429 0.13274143]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10050031 0.73753332 0.4        0.25396825]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09977122 0.63558591 0.5   

ETR
KNN
SVR
SR
fitting
predicting
[0.09945529 0.93260111 0.5        0.32539683]
fold number:  2
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09993753 0.59559481 0.5        0.19      ]
fold number:  3
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09988558 0.79830796 0.46666667 0.19047619]
fold number:  4
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10050016 0.5022824  0.5        0.18      ]
fold number:  5
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10008143 0.70497117 0.2        0.2       ]
fold number:  6
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10032596 0.91411751 0.5        0.39      ]
fold number:  7
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10025024 0.70342904 0.57142857 0.18      ]
fold number:  8
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.1005     0.53302332 1.42857143 0.75968443]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10001448 0.69411152 0.5        0.19276819]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09993967 0.77581631 1.    

ETR
KNN
SVR
SR
fitting
predicting
[0.09990462 0.75182258 0.46666667 0.27440762]
fold number:  2
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09970055 0.8049073  0.4        0.33300236]
fold number:  3
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.1000304  0.82769169 0.71428571 0.36291677]
fold number:  4
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09990098 0.62953608 0.4        0.31333333]
fold number:  5
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09933113 0.84157489 0.3        0.25975283]
fold number:  6
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10050037 0.48855209 0.57142857 0.29223801]
fold number:  7
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10044001 0.80775517 0.71428571 0.38589234]
fold number:  8
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09963914 0.46763918 0.46666667 0.32049817]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.09970837 0.63222078 0.46666667 0.1       ]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.10000123 0.80703892 2.    

...

ETR
KNN
SVR
SR
fitting
predicting
[1.14747019 1.06701369 1.57142857 0.882     ]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.10586231 2.08488719 1.6        2.        ]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.87893618 0.76383606 1.5        0.79      ]
dtr: mre:  0.5354798129507137  mae:  1.8681497175141242  rmse:  4.340641738710337
test dtr: mre:  0.5666042734178067  mae:  2.179432624113475  rmse:  4.764837664492422
rfr: mre:  0.7078512783153942  mae:  2.167089719748151  rmse:  4.634545599891775
test rfr: mre:  0.5380620712107579  mae:  1.7882383832499809  rmse:  3.723105891534824
etr: mre:  0.7543712480373382  mae:  2.160379108147738  rmse:  4.550624909096917
test etr: mre:  0.6314568880406417  mae:  1.871923337626645  rmse:  3.676754607413276
knn: mre:  0.7905933457539345  mae:  2.2263559322033895  rmse:  4.500576367546037
test knn: mre:  0.5993983576184683  mae:  1.901063829787234  rmse:  3.665512953119852
svr: mre:  0.6586726048061194  mae:  2.2

ETR
KNN
SVR
SR
fitting
predicting
[1.10017856 1.07075329 1.57142857 0.9004    ]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.09996268 0.89896752 1.4        0.75133333]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.0995     1.14276302 1.4        0.77777778]
dtr: mre:  0.5586681451130513  mae:  1.9000706214689262  rmse:  4.348698681887028
test dtr: mre:  0.6664343351859902  mae:  2.4445035460992908  rmse:  5.789625013940297
rfr: mre:  0.7012681374324894  mae:  2.1619615797969773  rmse:  4.617551607916103
test rfr: mre:  0.531319291200681  mae:  1.7661850993009995  rmse:  3.802404961641643
etr: mre:  0.7609939238785819  mae:  2.1716261062906845  rmse:  4.554435063296418
test etr: mre:  0.6339879488852984  mae:  1.8820240002986584  rmse:  3.7464099880925836
knn: mre:  0.7942880091958673  mae:  2.23364406779661  rmse:  4.503828428574517
test knn: mre:  0.6104432922342423  mae:  1.9274468085106382  rmse:  3.7007126356845736
svr: mre:  0.6519758535623019  mae: 

ETR
KNN
SVR
SR
fitting
predicting
[1.07986798 1.46250405 1.57142857 0.79      ]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.09970142 0.95144046 1.5        0.76      ]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.02154039 1.29405679 1.6        0.787     ]
dtr: mre:  0.5832317505330398  mae:  1.907485875706215  rmse:  4.501965565419241
test dtr: mre:  0.6437177374875072  mae:  2.199822695035461  rmse:  4.787469472152782
rfr: mre:  0.7001979506548278  mae:  2.1536762895750075  rmse:  4.607639195377557
test rfr: mre:  0.5326328759908697  mae:  1.7821333956946686  rmse:  3.7955134598576628
etr: mre:  0.7608162000354349  mae:  2.1771956257658105  rmse:  4.56429370947801
test etr: mre:  0.6277616475099748  mae:  1.8884717855540643  rmse:  3.7814635681185913
knn: mre:  0.7881883243726133  mae:  2.2227966101694916  rmse:  4.491758895051419
test knn: mre:  0.5985285878339028  mae:  1.89063829787234  rmse:  3.7786411137583875
svr: mre:  0.6590301778955594  mae:  

ETR
KNN
SVR
SR
fitting
predicting
[0.72788337 1.2717281  1.4        0.76490909]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[0.74078477 1.17930152 1.53333333 0.72      ]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.09995444 1.18632603 1.7        0.84833333]
dtr: mre:  0.5542851287162913  mae:  1.8961158192090397  rmse:  4.429359704536142
test dtr: mre:  0.6197374713424545  mae:  2.1530141843971635  rmse:  5.060137501527734
rfr: mre:  0.6969135569806285  mae:  2.1471051106851617  rmse:  4.601170514563449
test rfr: mre:  0.5457952016114949  mae:  1.7913442085302436  rmse:  3.715579740064905
etr: mre:  0.7536772824537752  mae:  2.1586729084919263  rmse:  4.532046102351121
test etr: mre:  0.6348018432853848  mae:  1.8745809725298457  rmse:  3.657181033951916
knn: mre:  0.7942342990409262  mae:  2.240677966101695  rmse:  4.500928055553205
test knn: mre:  0.6114232153270722  mae:  1.9263829787234044  rmse:  3.7767639703064773
svr: mre:  0.6591795515552291  mae:

ETR
KNN
SVR
SR
fitting
predicting
[1.09969715 0.9076127  1.7        0.75591799]
fold number:  9
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.09950256 1.07909538 1.4        0.71      ]
fold number:  10
DTR
RFR
ETR
KNN
SVR
SR
fitting
predicting
[1.09990337 0.95898576 1.4        0.80952381]
dtr: mre:  0.5770271017459322  mae:  1.884816384180791  rmse:  4.394357195394255
test dtr: mre:  0.5410591857541581  mae:  2.1984042553191485  rmse:  5.019478452579028
rfr: mre:  0.6947512815736615  mae:  2.143741276038802  rmse:  4.5999001257084275
test rfr: mre:  0.542971553913538  mae:  1.813910188298283  rmse:  3.68467848832992
etr: mre:  0.7593316845463343  mae:  2.165141615408162  rmse:  4.55158268366313
test etr: mre:  0.6179464750717031  mae:  1.8733952042939155  rmse:  3.5718751956648744
knn: mre:  0.7935147495883083  mae:  2.2362711864406775  rmse:  4.496198761793909
test knn: mre:  0.6023222541591795  mae:  1.9185106382978723  rmse:  3.6664666784112554
svr: mre:  0.6568785670749946  mae:  2.